# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучаем общую информацию

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

In [3]:
import pandas as pd
df = pd.read_csv('data.csv')
df.tail(20) # выводим последние 20 строк

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21506,1,-1556.249906,33,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба
21507,1,-79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости


In [2]:
df.info()
print('Размер таблицы:',df.shape)
print('__________')
print('Кол-во пропущенных значенй')
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Размер таблицы: (21525, 12)
__________
Кол-во пропущенных значенй


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [3]:
nan = df[df.isna().any(axis=1)] #показать все NaN

In [4]:
print(nan['purpose'].value_counts()) #показать кол-во уникальных значений в df 'nan'

на проведение свадьбы                     92
сыграть свадьбу                           81
свадьба                                   76
строительство собственной недвижимости    75
операции с жильем                         74
покупка недвижимости                      72
операции со своей недвижимостью           71
покупка жилья для семьи                   71
операции с коммерческой недвижимостью     70
ремонт жилью                              70
покупка коммерческой недвижимости         67
покупка жилья для сдачи                   65
недвижимость                              62
операции с недвижимостью                  61
покупка жилой недвижимости                61
жилье                                     60
строительство недвижимости                59
автомобили                                57
заняться высшим образованием              56
заняться образованием                     55
сделка с подержанным автомобилем          54
на покупку своего автомобиля              53
свой автом

In [5]:
#Поиск уникальных значений
print('----------------------------------')
print('Уникальные значени в children\n', df['children'].unique())
print('\n')
print('----------------------------------')
a = df['dob_years'].unique()
print('Уникальные значени в dob_years\n', sorted(a, reverse = False)) #отсортировано по возрастанию
print('\n')
print('----------------------------------')
print('Уникальные значени в education\n', df['education'].unique())
print('\n')
print('----------------------------------')
print('Уникальные значени в income_type\n', df['income_type'].unique())
print('\n')
print('----------------------------------')
print('Уникальные значени в family_status\n', df['family_status'].unique())
print('\n')
print('----------------------------------')
print('Уникальные значени в purpose\n', df['purpose'].unique()) 

----------------------------------
Уникальные значени в children
 [ 1  0  3  2 -1  4 20  5]


----------------------------------
Уникальные значени в dob_years
 [0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


----------------------------------
Уникальные значени в education
 ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


----------------------------------
Уникальные значени в income_type
 ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


----------------------------------
Уникальные значени в family_status
 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не 

**Вывод**

Присутвуют:

1. Отрицательные значения в days_employed;
0. Отрицательные значения в 'children';
0. dob_years - имееться нулевое значение;
0 . 'children' = 20, слишком много значений, чтобы быть правдой, да и возраст многих клиентов не велик, чтобы подтвердить наличие 20 детей.
0. days_employed и total_income присутсвует равное кол-во НЕслучаных NaN:
    
    Гипотезы появления NaN:
        - небольшой размер кредита;
        - положительная кредитная история;
        - имущественный залог;
        - выдача кредита без справки о доходах.

0. Education - есть проблемы с upper и low case;
0. Поблемы с типами данных: 
    days_employed  - float -> int
    total_icome - float -> int
0. вероятно есть дубли;
0. purpose - много однотипных категорий.
    

## Шаг 2. Предобрабатываем данные

### Обработка пропусков

основываясь на сформированные гипотезы, можно сказать, что пропуски - "Неслучайные".
1. Создать категории:
    - [x] нулевое значение в 'dob_years' 
    - [x] 'children', 'days_employed' поставить в модуль abs() - позволит убрать отриц. значения;
    - [x] разбить df без NaN по типу занятости найдя уникальные значения;
    - [x] посчитать медианны в 'days_employed' и 'total income';
    - [x] разбить пропуски по типу занятости;
    - [x] проставить медианные значения в 'days_employed' и 'total income';
    - [x] 'days_employed', 'total_income' изменить тип данных на int;

In [6]:
df.loc[df['children'] == 20, 'children'] = 2 #меняем опечатки 20 на 2 в 'children' 
df['children'] = df['children'].abs() # переводим отрицательные числа в положитлеьные, применяя модуль для числа
df['days_employed'] = df['days_employed'].abs() # переводим отрицательные числа в положитлеьные, применяя модуль для числа
# df.tail(20)
print('Кол-во детей у клиентов:\n',df['children'].value_counts())
print('Уникальные значени в income_type\n', df['income_type'].unique())

Кол-во детей у клиентов:
 0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64
Уникальные значени в income_type
 ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [7]:
# точное вычисление трудового стажа нам с выводами не поможет, но приведём таблицу к нормальному виду данных.
# для удобства, узнаем сколько это в годах. Обынчо в РФ, в год 247 рабочих дней в год
# предположим что пенсионер или безработный за свой трудоспособный возраст имееют стаж 30 лет. А это 30 * 247 = 7410 дней.

df.loc[(df['days_employed'] > 7410, 'days_employed')] = 7410

days_employed_max = df.groupby('income_type').agg({'days_employed' : 'median'})
days_employed_min = df.groupby('income_type').agg({'days_employed' : 'min'})

days_employed_max ['years'] = days_employed_max['days_employed'] / 247
days_employed_min ['years'] = days_employed_min ['days_employed'] / 247
print(days_employed_max)
print(days_employed_min)

# избавились от экстремумов. Откуда они взялись в изначальном df, нужно поинтересоваться у заказчика.


                 days_employed      years
income_type                              
безработный        7410.000000  30.000000
в декрете          3296.759962  13.347206
госслужащий        2689.368353  10.888131
компаньон          1547.382223   6.264705
пенсионер          7410.000000  30.000000
предприниматель     520.848083   2.108697
сотрудник          1574.202821   6.373291
студент             578.751554   2.343124
                 days_employed      years
income_type                              
безработный        7410.000000  30.000000
в декрете          3296.759962  13.347206
госслужащий          39.954170   0.161758
компаньон            30.195337   0.122248
пенсионер          7410.000000  30.000000
предприниматель     520.848083   2.108697
сотрудник            24.141633   0.097739
студент             578.751554   2.343124


In [8]:
# Найдем по каждому 'income_type' - типу занятости, медианный возраст
dob_years_median = df.groupby('income_type').agg({'dob_years' : 'median'})
# проставим медианное значение возраста, вместо нулей.
# если возраст = 0, то проставить медианое значение, в соответсвии с типом занятости

for i in df.income_type.unique():
     df.loc[df['income_type'] == i, 'dob_years'] = df.loc[df['income_type'] == i, 'dob_years'].replace(0 , dob_years_median.loc[i, 'dob_years'])

In [9]:
#Найдём по каждому типу занятости медиану 'days_employed' - общий трудовой стаж в днях
#'сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'  'предприниматель' 'студент' 'в декрете'

days_employed_median = df.groupby('income_type').agg({'days_employed' : 'median'})
print(days_employed_median)
# вместо NaN в 'days_employed' подставим значения в таблицу
for i in df.income_type.unique():
    df.loc[df['income_type'] == i, 'days_employed'] = df.loc[df['income_type'] == i, 'days_employed'].fillna(days_employed_median.loc[i, 'days_employed'])  

                 days_employed
income_type                   
безработный        7410.000000
в декрете          3296.759962
госслужащий        2689.368353
компаньон          1547.382223
пенсионер          7410.000000
предприниматель     520.848083
сотрудник          1574.202821
студент             578.751554


In [10]:
# Найдём по каждому типу занятости медиану 'total_income' - ежемесячный доход
total_income_median = df.groupby('income_type').agg({'total_income' : 'median'})

# вместо NaN в 'total_income' подставим значения в таблицу
for i in df.income_type.unique():
    df.loc[df['income_type'] == i, 'total_income'] = df.loc[df['income_type'] == i, 'total_income'].fillna(total_income_median.loc[i, 'total_income'])

#Выполним проверкуи убедимся что все значения заполнены
# df.tail(10)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Так как пропуски неслучайные, было принято решение заполнить их, опираясь на медианные значения по 'income_type' - типу занятости.

### Замена типа данных

In [11]:
df.days_employed= df.days_employed.astype('int')
df.total_income = df.total_income.astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для замены типа данных, был применён метот 'int' - целое число. Так как в кол-ве дней стажа и доход в месяц, нас интересуют целые числа, точность в данном случае будет излишней.

### Обработка дубликатов

- [x] привести строки в колонках 'education' и 'family_status' к нижнему регистру
- [x] определить ключи
- [x] по ключам вывести дубликаты
- [x] удалить дубликаты с сохранением в новом df_without_dup

In [12]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

#Поиск уникальных значений
print('----------------------------------')
print('Уникальные значени в children\n', df['children'].unique())
print('\n')
print('----------------------------------')
a = df['dob_years'].unique()
print('Уникальные значени в dob_years\n', sorted(a, reverse = False)) #отсортировано по возрастанию
print('\n')
print('----------------------------------')
print('Уникальные значени в education\n', df['education'].unique())
print('\n')
print('----------------------------------')
print('Уникальные значени в income_type\n', df['income_type'].unique())
print('\n')
print('----------------------------------')
print('Уникальные значени в family_status\n', df['family_status'].unique())
print('\n')
print('----------------------------------')
print('Уникальные значени в purpose\n', df['purpose'].unique())

----------------------------------
Уникальные значени в children
 [1 0 3 2 4 5]


----------------------------------
Уникальные значени в dob_years
 [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


----------------------------------
Уникальные значени в education
 ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


----------------------------------
Уникальные значени в income_type
 ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


----------------------------------
Уникальные значени в family_status
 ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


----------------------------------
Уникальные значени в purpose
 ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование

In [13]:
key = ['children', 'days_employed', 'dob_years', 'education', 'education_id', 'family_status', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']

df.groupby(key)['days_employed'].count().sort_values(ascending=False).head(20)
df_whithout_dup = df.drop_duplicates(subset=key).copy()
print('df до удаления дублей:', df.shape)
print('df после удаления дублей:', df_whithout_dup.shape)

df до удаления дублей: (21525, 12)
df после удаления дублей: (21454, 12)


**Вывод**

Возможные причины появления дубликатов:
1. Повторное обращение:
     - т.е возможно это данные не выданных кредитов, а только обращения с анкетами;
     - обращения и выданные кредиты слиты в предоставленный df;
     - клиент обращался в несколько отделений банка, где оператор заполнял анкету заново
  
2. Ошибка внутри db банка;

### Лемматизация

In [14]:
df_whithout_dup.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,7410,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,7410,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


- [x] вывести отдельный df с уникальными значениями purpose
- [x] выполнить лемматизацию в этом по этому df
- [x] сгруппироват и проставить результаты в ddf_whithout_dup

In [15]:
purpose_category_values = (df_whithout_dup['purpose'].value_counts())
print(purpose_category_values)
print('------------------------------------')
print('Всего типов категорий:', purpose_category_values.count())


свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [16]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem(entire_input=False)
# лемматизируем строки в столбце 'purpose'
lemmas = m.lemmatize(' '.join(list(df_whithout_dup['purpose'])))
# print(lemmas)
# посчитаем кол-во повторений
from collections import Counter
Counter(lemmas)


# можно выделить следующие категории: недвижимость/жилье, автомобиль, образование, свадьба, строительство.

Counter({'покупка': 5897,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

**Вывод**

Наиболе встречающиеся категории: 
 - недвижимость/жилье
 - автомобиль
 - образование
 - свадьба
 - строительство

### Категоризация данных

In [17]:
# напишем функцию, которая перебер колонку 'purpose' и проставит категорию в новый столбец 'purpose_categories'
#в соответсвии с условиями категорий
def purpose_categories(purpose):
    lemmas = m.lemmatize(purpose)
    
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'строительство' in lemmas:
        return 'строительство'

df_whithout_dup['purpose_categories'] = df_whithout_dup['purpose'].apply(purpose_categories)

print(df_whithout_dup['purpose_categories'].value_counts())
# print(df_whithout_dup.head(10))

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_categories, dtype: int64


In [18]:
#удалим не нужный уже нам столбец со старыми категориями, и выведем первые 10 строк df
del df_whithout_dup['purpose']
df_whithout_dup.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_categories
0,1,7410,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,7410,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


**Вывод**

38 категорий сгурппировали в 5! При помощи лемматизации и категоризации, были присвоены соответсвующие категории для каждого клиента.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
#посчитаем была ли просрочка по кредиту, с разбивкой по кол-ву детей.
data_pivot_children = df_whithout_dup.pivot_table(index=['children'], values='debt', aggfunc='sum')
data_pivot_children['sum_loan'] = df_whithout_dup.groupby('children')['debt'].count()
data_pivot_children['ratio'] = data_pivot_children['debt']/ data_pivot_children['sum_loan']
print(data_pivot_children.sort_values(by=['debt'], ascending=False))
print('----------------------------------')

# сравним доли клиентов, у которых есть дети и нет детей. Выведем результат.
children_destiny = (len(df_whithout_dup.loc[df_whithout_dup['children'] > 0]))/(len(df_whithout_dup.loc[df_whithout_dup['children'] == 0]))
print('Доля клиентов с детьми: {:.2f}' .format(children_destiny))

          debt  sum_loan     ratio
children                          
0         1063     14091  0.075438
1          445      4855  0.091658
2          202      2128  0.094925
3           27       330  0.081818
4            4        41  0.097561
5            0         9  0.000000
----------------------------------
Доля клиентов с детьми: 0.52


**Вывод**

Зависимость между наличием детей и возватом кредита в срок есть, но она неоднозначная. Чаще всего просрочка по кредиту образуеся у клиентов с детьми.

Можно сделать несколько дополнительных гипотез:
1. Страхуя свои риски, банк старается больше кредитовать тех, у кого отсутсвуют дети;
0. Возможно заёмщики с детьми, берут более дорогостоящие кредиты, которвые сложно отдать, чем те кто без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
# посчтаем была ли просрочка по кредиту, в зависимости от семейного положения
# вывдем pivot и добавим столбцы с суммарным кол-вом заёмщиков по семейному статусу и посчитам долю просрочки по возврату кредита
data_pivot_family_status = df_whithout_dup.pivot_table(index = ['family_status'], values = 'debt', aggfunc = 'sum')
data_pivot_family_status['sum_loan'] = df_whithout_dup.groupby('family_status')['debt'].count()
data_pivot_family_status['ratio'] = data_pivot_family_status['debt'] / data_pivot_family_status['sum_loan']
data_pivot_family_status.sort_values(by=['ratio'], ascending=False)

,debt,sum_loan,ratio
family_status,,,
не женат / не замужем,274,2810,0.097509
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693


**Вывод**

Зависимость между семейным положением и возвратом кредита присутствует. Заёмщики не состоявшие ни разу в законном браке, чаще имеют просрочки по платежам. А заёмщики в разводе или овдовевшие, имеют меньший процент по просрочкам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
# для категоризации заёмщиков на категории, напишем функцию с целочисленным делением на 50 000 - чтобы выборка была репрезентативной, 
# в новый столбец проставим 'total_income_id'

def total_income_category(value):
    if value >= 500000:
        return 4
    else:
        value = value // 200000
        return value

df_whithout_dup['total_income_id'] = df_whithout_dup['total_income'].apply(total_income_category)
df_whithout_dup['total_income_id'].value_counts()

data_pivot_income = df_whithout_dup.pivot_table(index = ['total_income_id'], values = 'debt', aggfunc = 'sum')
data_pivot_income['sum_loan'] = df_whithout_dup.groupby('total_income_id')['debt'].count()
data_pivot_income['ratio'] = data_pivot_income['debt']/ data_pivot_income['sum_loan']
print(data_pivot_income.sort_values('ratio', ascending = False))

                 debt  sum_loan     ratio
total_income_id                          
0                1383     16387  0.084396
1                 327      4538  0.072058
4                  14       222  0.063063
2                  17       307  0.055375


**Вывод**

Зависимость между уровнем дохода и возвратом кредита в срок - отсутсвует

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
data_pivot_purpose = df_whithout_dup.pivot_table(index = ['purpose_categories'], values = 'debt', aggfunc = 'sum')
data_pivot_purpose['sum_loan'] = df_whithout_dup.groupby('purpose_categories')['debt'].count()
data_pivot_purpose['ratio'] = data_pivot_purpose['debt']/data_pivot_purpose['sum_loan']
print(data_pivot_purpose.sort_values('ratio', ascending=False))

                    debt  sum_loan     ratio
purpose_categories                          
автомобиль           403      4306  0.093590
образование          370      4013  0.092200
свадьба              186      2324  0.080034
недвижимость         782     10811  0.072334


**Вывод**

Чаще всего просрочка проиходит по категории 'автомобиль' и 'образование', а меньше всего в категории 'недвижимость' и 'свадьба'.

Также в категории 'недвижимость' почти в 2 раза превышает кол-во выданных кредитов, но при этом задолженность по возвратам самая низкая из представленных категорий

## Шаг 4. Общий вывод

1. Нельзя однозначно сказать, что кол-во детей влияет на возникновение просрочки по кредиту, так как выборка нерепрезентативная, нужно запросить больше данных по кол-ву выданных кредитов, заёмщикам с детьми.
0. Семейное положение влияет на возникновение задолженности по кредиту. Заёмщики состоявшие или состоящие в браке, имеют меньше просрочек по кредиту, чем тем кто не состоял в браке. При том разведённые или овдовевшие образуют меньшую задолженность по кредитам.
0. Наименьшее кол-во просрочек по выданных кредитам, присутсвует в категориях 'недвижимость' и 'свадьба', а наибольшее кол-во просрочек в категориях - 'автомобиль' и 'образование'. 

> Возможно столь высокая просрочка по категории 'автомобиль' объяснятеся тем, что личный автомобиль быстро теряет в цене и требует много оперативных затрат. В 'образовании' студенты доверяют себя организациям и полностью полагаються на них, веря в обещенное трудоустройсво по окончанию обучения. Но это уже совсем другое исследование....  